In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the GPT-J model and tokenizer
model_name = "EleutherAI/gpt-j-6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Using device: cuda


In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

: 

In [ ]:
# Sample dataset for demonstration
dataset = [
    {
        "prompt": "Let's start with Django. How does Django manage user sessions?",
        "response": "Django uses a session framework to store data on a per-user basis. It generates a session ID that's stored in the user's browser, while the associated session data is stored on the server, typically in the database or a cache."
    },
    {
        "prompt": "Good. Can you elaborate on how Django uses middleware in managing sessions?",
        "response": "Sure. Middleware in Django processes requests before they reach the view and after they've been processed by the view. For sessions, Django has middleware that reads the session data from the request and loads it, then saves any changes to the session before sending the response back to the client."
    },
    {
        "prompt": "That's correct. Now, explain how the Django ORM simplifies interactions with the database.",
        "response": "The Django ORM lets developers interact with the database using Python code instead of writing raw SQL. It provides models that map directly to database tables, which makes querying, creating, and updating records easy and database-agnostic."
    },
    {
        "prompt": "What are some limitations or challenges with using Django ORM?",
        "response": "One challenge is with complex queries. While Django ORM covers most CRUD operations well, it can become less efficient for complex joins or aggregations. In those cases, developers sometimes need to use raw SQL for better performance."
    },
    {
        "prompt": "That makes sense. Moving to Flask, what are blueprints and how would they benefit a larger application?",
        "response": "Blueprints in Flask help organize application functionality by allowing different sections of an app to be separated into modular components. For example, in a large app, you could have separate blueprints for authentication, user management, and content, which keeps the codebase more organized and scalable."
    },
    {
        "prompt": "If we use Flask for an API-based application, how would you handle HTTP methods and route organization?",
        "response": "In Flask, you can specify HTTP methods directly on routes by setting the `methods` argument. For example, using `@app.route('/login', methods=['POST'])` ensures that the route only accepts POST requests. Routes can be further organized by grouping them within blueprints, which keeps related routes in one place."
    },
    {
        "prompt": "Let's discuss environments. How can Flask manage different environments like development and production?",
        "response": "Flask can handle multiple environments through configuration objects or files. By setting the `FLASK_ENV` environment variable to 'development' or 'production', you can load different settings, such as enabling debug mode in development and setting specific database URIs for each environment."
    },
    {
        "prompt": "Can you explain Django signals and provide an example of when they'd be useful in a project?",
        "response": "Django signals are used to trigger actions when certain events occur in the application. For instance, you might use a signal to create a user profile automatically after a new user registers. This decouples the user creation process from the profile creation process, which keeps the code modular."
    },
    {
        "prompt": "Great example. Now, how would you approach file uploads in Flask?",
        "response": "In Flask, file uploads are handled through `request.files`. First, you configure an upload folder using `app.config['UPLOAD_FOLDER']`, then access the file via `request.files['file']`. Finally, you can save it to the server using the `save` method."
    },
    {
        "prompt": "Lastly, in both Django and Flask, how would you implement user authentication in a custom way?",
        "response": "In Django, custom authentication can be done by creating a custom user model and setting `AUTH_USER_MODEL` in `settings.py`. In Flask, custom authentication typically involves creating a login route that checks credentials against a database, using JWT tokens or sessions to manage authenticated states."
    }
]

# Function to prepare dataset for training
def prepare_dataset(dataset):
    texts = []
    for entry in dataset:
        text = f"Interviewer: {entry['prompt']}\nCandidate: {entry['response']}\n"
        texts.append(text)
    return Dataset.from_dict({"text": texts})

# Convert to Hugging Face Dataset format
hf_dataset = prepare_dataset(dataset)

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

In [ ]:
# Define Training Arguments with Logging
training_args = TrainingArguments(
    output_dir="./gptj-interview",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",         # Directory for storing logs
    logging_steps=10,             # Log every 10 steps
    report_to="tensorboard"       # Report logs to TensorBoard
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Fine-Tuning with Logging
trainer.train()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model and tokenizer
model_name = "./gptj-interview"  # Path to the fine-tuned model directory
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Function to simulate the interview
def interview_simulation(model, tokenizer):
    # Initial context to set the tone for the interview
    context = "Candidate: Hello, I'm ready for the interview.\nInterviewer: Welcome! Let's start with some questions about Django and Flask.\n"
    
    # First question to start the interview
    interviewer_question = "To start, can you describe how Django manages user sessions?"
    print(f"Interviewer: {interviewer_question}")
    context += f"Interviewer: {interviewer_question}\n"
    
    # Start the interactive loop
    while True:
        # Candidate's input
        candidate_response = input("Candidate: ")
        context += f"Candidate: {candidate_response}\n"
        
        # Prepare context for next interviewer question
        context += "Interviewer: "
        
        # Tokenize and generate the next question based on the context
        inputs = tokenizer(context, return_tensors="pt")
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_length=512,
                pad_token_id=tokenizer.eos_token_id,
                num_return_sequences=1,
                no_repeat_ngram_size=3,
                temperature=0.7  # Control creativity for follow-ups
            )
        
        # Decode the generated question
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        next_interviewer_question = generated_text.split("Interviewer:")[-1].strip().split("Candidate:")[0].strip()
        
        # Print and update context
        print(f"Interviewer: {next_interviewer_question}")
        context += f"{next_interviewer_question}\n"

        # Exit condition for the loop
        if candidate_response.lower() in ["exit", "quit"]:
            print("Interviewer: Thank you for your time. This concludes the interview.")
            break

# Run the interview simulation
interview_simulation(model, tokenizer)